In [1]:
# imports and setup 

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, cross_val_predict, cross_val_score, KFold

from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout


Using TensorFlow backend.


In [2]:
def get_data_from_file(file_path):
    df = pd.read_csv(file_path)
        
    x = organize_pixel_values(df)   
    y = np.array(df.m_label)
    
    return x, y

In [26]:
def organize_pixel_values(raw_df):
    df = raw_df.copy();
    for i in range(12):
        df.drop(df.columns[[0]], axis=1, inplace=True)

    ret = []
    column_vals = df.columns
    df_matrix = df.to_numpy()
    
    for row in range(len(df_matrix)):
        pixel_matrix = np.zeros((20, 20))
        for col in range(len(df_matrix[0])):
            header = column_vals[col]
            splitHeader = header[1:].split('c')
            pixelRow = splitHeader[0]
            pixelCol = splitHeader[1]
            pixel_matrix[int(pixelRow)][int(pixelCol)] = df_matrix[row][col] / 255.0
        ret.append(pixel_matrix)
    
    ret = np.stack(ret)
    ret = np.reshape(ret, (-1, 20, 20, 1))
    
    return ret

In [4]:
def get_maps_from_labels(input_arr):  
    val_to_ix = { val:i for i,val in enumerate(np.unique(input_arr)) }
    ix_to_val = { i:val for i,val in enumerate(np.unique(input_arr)) }
    
    return val_to_ix, ix_to_val

In [5]:
def reconcile_fonts(font1, font2):
    x1 = font1.x
    y1 = font1.y_raw
    x2 = font2.x
    y2 = font2.y_raw
    
#     print(len(x1))
#     print(len(y1))
#     print(len(x2))
#     print(len(y2))

    intersection = np.intersect1d(y1, y2, assume_unique=False, return_indices=False)
    
#     print(intersection)
    
    x1_fixed = []
    y1_fixed = []
    for i in range(len(y1)):
        if (y1[i] in intersection):
            x1_fixed.append(x1[i])
            y1_fixed.append(y1[i])

    x2_fixed = []
    y2_fixed = []
    for i in range(len(y2)):
        if (y2[i] in intersection):
            x2_fixed.append(x2[i])
            y2_fixed.append(y2[i])
    
#     print(len(x1_fixed))
#     print(len(y1_fixed))
#     print(len(x2_fixed))
#     print(len(y2_fixed))
    
    font1_fixed = Font(np.asarray(x1_fixed), np.asarray(y1_fixed))
    font2_fixed = Font(np.asarray(x2_fixed), np.asarray(y2_fixed))
    
#     print(font1.unique_char_count)
#     print(font1.unique_char_count)
    
    return font1_fixed, font2_fixed

In [6]:
class Font:
    x = None
    y = None
    y_raw = None
    
    unique_char_count = 0
    val_to_ix = None
    ix_to_val = None
    
    def __init__(self, data, labels):
        self.x = data
        self.y_raw = labels
        self.val_to_ix, self.ix_to_val = get_maps_from_labels(self.y_raw)
        self.unique_char_count = len(self.val_to_ix)
        
        self.y = self.get_1_hot()
        
    def get_1_hot(self):
        ret = []
        for val in self.y_raw:
            arr = np.zeros(self.unique_char_count)
            arr[self.val_to_ix[val]] = 1;
            ret.append(arr)

        return np.array(ret)
    
    def display_attributes(self):
        print(self.ix_to_val[0])
        print(self.val_to_ix[33])

        print()

        print(self.unique_char_count)

        print()

        print(self.x.shape)
        print(self.y.shape)

In [27]:
x, y = get_data_from_file('fonts/ARIAL.csv')
arial = Font(x, y)

In [8]:
arial.display_attributes()

33
0

3098

(26237, 20, 20, 1)
(26237, 3098)


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(arial.x, arial.y, random_state=1, test_size=0.8)

In [ ]:
model = Sequential()

model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(20, 20, 1)))
model.add(MaxPooling2D((2, 2), padding="same"))
model.add(Conv2D(64, kernel_size=3, activation='relu'))
model.add(MaxPooling2D((2, 2), padding="same"))
model.add(Flatten())
model.add(Dropout(.1))
model.add(Dense(64, activation='relu'))
model.add(Dense(arial.unique_char_count, activation='softmax'))

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=20)

### Evaluate the network using cross validation (splitting data into training/testing). What is its accuracy? ###

**Training and testing on Arial, our model gives us an accuracy of around 47% after 20 epochs**

In [ ]:
model2 = Sequential()

model2.add(Conv2D(128, kernel_size=3, activation='relu', input_shape=(20, 20, 1)))
model2.add(MaxPooling2D((2, 2), padding="same"))
model2.add(Conv2D(128, kernel_size=3, activation='relu'))
model2.add(MaxPooling2D((2, 2), padding="same"))
model2.add(Flatten())
model2.add(Dropout(.1))
model2.add(Dense(128, activation='relu'))
model2.add(Dense(arial.unique_char_count, activation='softmax'))

In [ ]:
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model2.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=20)

### Create and train a different network topology (add more convolution/dropout layers, explore other types/sizes of layer). Try to find a topology that works better than the one described above. ###

**I took a very simple approach with my second model, I just doubled the number of perceptrons of the convolutional and dense layers. This new model gives us a modest increase of accuracy (48% after 20 epochs).  Each epoch took between 2 and 2.5x as long to run though, so it's possible that the increased accuracy would not be worth the time/computing cost in practice**

In [ ]:
model3 = Sequential()

model3.add(Conv2D(64, kernel_size=5, activation='relu', input_shape=(20, 20, 1)))
model3.add(MaxPooling2D((2, 2), padding="same"))
model3.add(Conv2D(64, kernel_size=5, activation='relu'))
model3.add(MaxPooling2D((2, 2), padding="same"))
model3.add(Flatten())
model3.add(Dropout(.1))
model3.add(Dense(64, activation='relu'))
model3.add(Dense(arial.unique_char_count, activation='softmax'))

In [ ]:
model3.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model3.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=20)

In [ ]:
model4 = Sequential()

model4.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(20, 20, 1)))
model4.add(MaxPooling2D((2, 2), padding="same"))
model4.add(Conv2D(64, kernel_size=3, activation='relu'))
model4.add(MaxPooling2D((2, 2), padding="same"))
model4.add(Conv2D(64, kernel_size=3, activation='relu'))
model4.add(MaxPooling2D((2, 2), padding="same"))
model4.add(Flatten())
model4.add(Dropout(.1))
model4.add(Dense(64, activation='relu'))
model4.add(Dense(arial.unique_char_count, activation='softmax'))

In [ ]:
model4.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model4.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=20)

### Third and Fourth Models ###

**I did a third and forth model mostly for my enjoyment (since the second model was already an improvment over the first).  The third model I changed the kernel size to 5x5 and it underpreformed with 47% accuracy.  The fourth model I added one additional convolutional layer * max pooling layer and (to my surprise) it dramatically underperformed at 43% accuracy.**

In [9]:
x, y = get_data_from_file('fonts/TIMES.csv')
times = Font(x, y)

In [10]:
times.display_attributes()

33
0

3087

(12730, 20, 20, 1)
(12730, 3087)


In [11]:
fixedArial, fixedTimes = reconcile_fonts(arial, times)

In [12]:
fixedArial.display_attributes()

33
0

3087

(26180, 20, 20, 1)
(26180, 3087)


In [13]:
fixedTimes.display_attributes()

33
0

3087

(12730, 20, 20, 1)
(12730, 3087)


In [ ]:
model_font_comparison = Sequential()

model_font_comparison.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(20, 20, 1)))
model_font_comparison.add(MaxPooling2D((2, 2), padding="same"))
model_font_comparison.add(Conv2D(64, kernel_size=3, activation='relu'))
model_font_comparison.add(MaxPooling2D((2, 2), padding="same"))
model_font_comparison.add(Flatten())
model_font_comparison.add(Dropout(.1))
model_font_comparison.add(Dense(64, activation='relu'))
model_font_comparison.add(Dense(fixedArial.unique_char_count, activation='softmax'))

In [ ]:
model_font_comparison.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model_font_comparison.fit(fixedArial.x, fixedArial.y, validation_data=(fixedTimes.x, fixedTimes.y), epochs=20)

### Test the accuracy of your network with character inputs from a DIFFERENT font set. How does it perform? ###

**I trained with Arial and tested on Times New Roman and, unsurprisingly we took fairly significant hit ot our accuracy - down all the way to 31%.**


In [14]:
x, y = get_data_from_file('fonts/COMIC.csv')
comic_sans = Font(x, y)

In [15]:
comic_sans.display_attributes()

33
0

597

(2388, 20, 20, 1)
(2388, 597)


In [16]:
fixedArial, fixedComic = reconcile_fonts(fixedArial, comic_sans)

In [17]:
fixedTimes, fixedComic = reconcile_fonts(fixedTimes, fixedComic)

In [18]:
fixedArial.display_attributes()

33
0

579

(16872, 20, 20, 1)
(16872, 579)


In [19]:
fixedTimes.display_attributes()

33
0

579

(4034, 20, 20, 1)
(4034, 579)


In [20]:
fixedComic.display_attributes()

33
0

579

(2316, 20, 20, 1)
(2316, 579)


In [21]:
model_2fonts = Sequential()

model_2fonts.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(20, 20, 1)))
model_2fonts.add(MaxPooling2D((2, 2), padding="same"))
model_2fonts.add(Conv2D(64, kernel_size=3, activation='relu'))
model_2fonts.add(MaxPooling2D((2, 2), padding="same"))
model_2fonts.add(Flatten())
model_2fonts.add(Dropout(.1))
model_2fonts.add(Dense(64, activation='relu'))
model_2fonts.add(Dense(fixedArial.unique_char_count, activation='softmax'))

W0717 09:22:06.313472 4551812544 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0717 09:22:06.360015 4551812544 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0717 09:22:06.374168 4551812544 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0717 09:22:06.407280 4551812544 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0717 09:22:06.444809 4551812544 deprecation_wrapper.py:119] From /usr/loca

In [22]:
model_2fonts.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

W0717 09:22:06.523973 4551812544 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0717 09:22:06.555028 4551812544 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



In [23]:
model_2fonts.fit(np.concatenate([fixedArial.x, fixedTimes.x]), \
                 np.concatenate([fixedArial.y, fixedTimes.y]), \
                 validation_data=(fixedComic.x, fixedComic.y), epochs=20)

W0717 09:22:06.840777 4551812544 deprecation.py:323] From /usr/local/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 20906 samples, validate on 2316 samples
Epoch 1/20
20906/20906 [==============================] - 11s 542us/step - loss: 3.0091 - acc: 0.4680 - val_loss: 3.8140 - val_acc: 0.2396
Epoch 2/20
20906/20906 [==============================] - 9s 451us/step - loss: 1.1780 - acc: 0.6721 - val_loss: 3.2768 - val_acc: 0.3169
Epoch 3/20
20906/20906 [==============================] - 9s 436us/step - loss: 0.8557 - acc: 0.7363 - val_loss: 3.2978 - val_acc: 0.3342
Epoch 4/20
20906/20906 [==============================] - 9s 438us/step - loss: 0.7106 - acc: 0.7681 - val_loss: 3.2444 - val_acc: 0.3605
Epoch 5/20
20906/20906 [==============================] - 9s 444us/step - loss: 0.6114 - acc: 0.7952 - val_loss: 3.3230 - val_acc: 0.3692
Epoch 6/20
20906/20906 [==============================] - 9s 445us/step - loss: 0.5451 - acc: 0.8145 - val_loss: 3.2099 - val_acc: 0.3916
Epoch 7/20
20906/20906 [==============================] - 12s 569us/step - loss: 0.4953 - acc: 0.8275 - val_loss: 3.4517 -

### Train your best network on inputs from the data from at least 2 different fonts. How does your accuracy compare to the 1-font case? What accuracy do you see when testing with inputs from a font you didn't train on?###

**I trained with Times New Roman and Arial and tested on Comic Sans.  The accuracy was dramatically better than our previous training on Times and testing on Arial (44% instead of 31%).  This might be because two datasets combined for training does a better job but it might also be because the comic sans dataset was much smaller than the previous two and so there are far fewer potential outputs (597 instead of 3087).

Test the accuracy of your network with character inputs from a DIFFERENT font set. How does it perform? (He means train on one font, test on another)

Train your best network on inputs from the data from at least 2 different fonts. How does your accuracy compare to the 1-font case? What accuracy do you see when testing with inputs from a font you didn't train on?

Take a look at some of the characters that have been misclassified. Do you notice any patterns? The network only produces the relative probabilities that the input is any of the possible characters. Can you find examples where the network is unsure of the result?